## 0. Load

In [57]:
import datetime as dt
import polars as pl
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [79]:
zones_df = (
  pl.scan_csv("taxi+_zone_lookup.csv")
)
display(zones_df.collect().head(3))

LocationID,Borough,Zone,service_zone
i64,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""


In [52]:
taxi_trip_df = (
  pl.scan_csv("green_tripdata_2019-09.csv")
  .with_columns([
    pl.col(["lpep_pickup_datetime", "lpep_dropoff_datetime"]).str.to_datetime(),
    # pl.col("lpep_pickup_datetime").dt.date().alias("lpep_pickup_date")
  ])
)

## 1. Questions

In [53]:
taxi_trip_df.first().collect()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
i64,datetime[μs],datetime[μs],str,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,str,f64,f64,i64,i64,f64
2,2019-09-01 00:10:53,2019-09-01 00:23:46,"""N""",1,65,189,5,2.0,10.5,0.5,0.5,2.36,0.0,null,0.3,14.16,1,1,0.0


In [54]:
taxi_trip_df.null_count().collect()

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
59143,0,0,59143,59143,0,0,59143,0,0,0,0,0,0,449063,0,0,59143,59143,59143


In [55]:
## Question 3
date_filter = dt.date(2019, 9, 18)
(
  taxi_trip_df
  .filter(
    (pl.col("lpep_pickup_datetime").dt.date() == date_filter) &
    (pl.col("lpep_dropoff_datetime").dt.date() == date_filter)
  )
  .select([
    pl.col("lpep_dropoff_datetime").count().alias('n_rows'),
  ])
  .collect()
)

n_rows
u32
15612


In [56]:
## Question 4
(
  taxi_trip_df
  .with_columns([pl.col("lpep_pickup_datetime").dt.date().alias("lpep_pickup_date")])
  .group_by("lpep_pickup_date")
  .agg(pl.col("trip_distance").max().alias("largest_trip_distance"))
  .sort(by=["largest_trip_distance"], descending=True)
  .collect()
  .head(10)
)

lpep_pickup_date,largest_trip_distance
date,f64
2019-09-26,341.64
2019-09-21,135.53
2019-09-16,114.3
2019-09-28,89.64
2019-09-24,82.12
2019-09-18,70.28
2019-09-10,69.67
2019-09-27,68.41
2019-09-02,61.69


In [77]:
## Question 5
(
  taxi_trip_df
  .filter(
    (pl.col("lpep_pickup_datetime").dt.date() == dt.date(2019, 9, 18))
  )
  .join(
    zones_df,
    left_on="PULocationID",
    right_on="LocationID",
    how="outer"
  )
  .group_by("Borough")
  .agg(pl.col("total_amount").sum().alias("borough_total_amount"))
  .sort(by=["borough_total_amount"], descending=True)
  .collect()
)

Borough,borough_total_amount
str,f64
"""Brooklyn""",96333.24
"""Manhattan""",92271.3
"""Queens""",78671.71
"""Bronx""",32830.09
"""Unknown""",728.75
"""Staten Island""",342.59
"""EWR""",0.0


In [88]:
## Question 6
(
  taxi_trip_df
  .join(
    zones_df,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
  )
  .join(
    zones_df,
    left_on="DOLocationID",
    right_on="LocationID",
    how="left",
    suffix="_dropoff"
  )
  .filter(pl.col("Zone") == "Astoria")
  # .first().collect()
  .group_by("Zone_dropoff")
  .agg(pl.col("tip_amount").max().alias("max_tip_amount"))
  .sort(by=["max_tip_amount"], descending=True)
  .collect()
  .head()
)

Zone_dropoff,max_tip_amount
str,f64
"""JFK Airport""",62.31
"""Woodside""",30.0
"""Kips Bay""",28.0
"""NV""",25.0
"""Astoria""",20.0
